<a href="https://colab.research.google.com/github/BM304/DA_Homeworks/blob/main/lesson2_hometask_belonenko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Задание


1.   Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти). Извлеките все описания этих вакансий
1.   Создайте аналогичный vacancy DataFrame только добавьте поле skills
1.   Переведите даты публикаций в datetime
1.   Постройте график опубликованных вакансий по датам
1.   Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
1.  Найдите те вакансии с использованием python, которые вам интересны
1.  Определите по полю skills какие навыки больше всего востребованы для этих 
вакансий
1.  Постройте график наиболее востребованных навыков для аналитика данных


>https://github.com/hhru/api


In [ ]:
# импорт модулей для работы
import requests
import json
from tqdm.auto import tqdm
from collections import defaultdict
import pickle
import pandas as pd
import plotly.express as px
from datetime import datetime
import numpy as np

import pylab
import matplotlib.dates as mdates




______________________________________________________________________
> 1.   Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти). Извлеките все описания этих вакансий
______________________________________________________________________


In [ ]:
# переменные для работы
# описание API https://github.com/hhru/api/blob/master/README.md 
sber = '3529'
nsk = 4
url = f'https://api.hh.ru/vacancies?employer_id={sber}&area={nsk}'


In [ ]:
# 1.   Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти). Извлеките все описания этих вакансий
# присваиваем в переменную результат запроса вакансий
res = requests.get(url)

In [ ]:
# 1.   Скачать все вакансии СБЕРа с ХХ (у апи есть ограничение в 2000, а также captcha подумайте как их можно обойти). Извлеките все описания этих вакансий
# смотрим полученные записи 
# P.S. не столкнулась с проблемой капчи, однако выставление параметров страницы уводило запрос в подчсет количества записей по вакансиям сбера в Новосибирске, прошлось скорректироваь запрос
# res.json()

______________________________________________________________________
> 2. Создайте аналогичный vacancy DataFrame только добавьте поле skills 
______________________________________________________________________


In [ ]:
# Выбираем данные по вакансиям, выводим несколько записей для проверки содержимого
vacancies = res.json()
v = vacancies.get('items')
v[:1]

[{'accept_temporary': False,
  'address': None,
  'adv_response_url': 'https://api.hh.ru/vacancies/66893967/adv_response?host=hh.ru',
  'alternate_url': 'https://hh.ru/vacancy/66893967',
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=66893967',
  'archived': False,
  'area': {'id': '4',
   'name': 'Новосибирск',
   'url': 'https://api.hh.ru/areas/4'},
  'contacts': None,
  'created_at': '2022-06-16T10:18:36+0300',
  'department': {'id': '3529-3529-stat', 'name': 'Сбер. Начало карьеры'},
  'employer': {'alternate_url': 'https://hh.ru/employer/3529',
   'id': '3529',
   'logo_urls': {'240': 'https://hhcdn.ru/employer-logo/3914324.jpeg',
    '90': 'https://hhcdn.ru/employer-logo/3914323.jpeg',
    'original': 'https://hhcdn.ru/employer-logo-original/868362.JPG'},
   'name': 'СБЕР',
   'trusted': True,
   'url': 'https://api.hh.ru/employers/3529',
   'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=3529'},
  'has_test': False,
  'id': '66893967',
  'i

In [ ]:
# уточняем тип v для дальнейшей конвертации
type(v)

list

In [ ]:
# пересобираем list v в dataframe и оставляем поля необходимые для выполения задания - даты и описания для выборки, ссылки на вакансии, регион, работодатель для самопроверки, смотрим что получилось 
df = pd.DataFrame(v, columns =['id','alternate_url','area','created_at','employer','name','snippet'], dtype = str) 
df.iloc[:2]


,id,alternate_url,area,created_at,employer,name,snippet
0,66893967,https://hh.ru/vacancy/66893967,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-06-16T10:18:36+0300,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Старший менеджер по обслуживанию,{'requirement': 'Хочешь стать экспертом в банк...
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01T07:46:13+0300,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен..."


In [ ]:
# создаем пустой датафрейм и в цикле заполнеям его значениями поля skills для каждой вакансии из найденного списка вакансий
df_skills = pd.DataFrame(columns =['id','skills'], dtype = str)

for i in range(len(df)):
    id = df.id[i]
    url_v = f'https://api.hh.ru/vacancies/{id}'
    res_skills = requests.get(url_v)
    vacancies_skills = res_skills.json()
    v_skills= vacancies_skills.get('key_skills')  
    new_row = {'id':id,'skills': v_skills}
    df_skills = df_skills.append(new_row, ignore_index=True)

# проверяем результат
df_skills.iloc[:2]


,id,skills
0,66893967,[{'name': 'Банковское кассовое обслуживание'}]
1,46580388,"[{'name': 'Публичные выступления'}, {'name': '..."


In [ ]:
# обьединяем датафреймы в 3 для выполнения задания
df_full = pd.merge(df, df_skills)

# проверяем результат
df_full.iloc[:2]

,id,alternate_url,area,created_at,employer,name,snippet,skills
0,66893967,https://hh.ru/vacancy/66893967,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-06-16T10:18:36+0300,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Старший менеджер по обслуживанию,{'requirement': 'Хочешь стать экспертом в банк...,[{'name': 'Банковское кассовое обслуживание'}]
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01T07:46:13+0300,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."



____________________________________________________________________________
> 3. Переведите даты публикаций в datetime

____________________________________________________________________________


In [ ]:
# Проводим обрезку строки даты создания до T для дальнейшей конвертации строки в дату, делаем перезапись
for i in range(len(df_full)):
    created_at = df.created_at[i].partition('T')[0]
    df_full.created_at[i] = datetime.strptime(created_at, "%Y-%m-%d") 


# проверяем результат
df_full.iloc[:2]

,id,alternate_url,area,created_at,employer,name,snippet,skills
0,66893967,https://hh.ru/vacancy/66893967,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-06-16 00:00:00,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Старший менеджер по обслуживанию,{'requirement': 'Хочешь стать экспертом в банк...,[{'name': 'Банковское кассовое обслуживание'}]
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01 00:00:00,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."


____________________________________________________________________________

> 4. Постройте график опубликованных вакансий по датам
____________________________________________________________________________

In [ ]:
# подсчитываем количество публикаций на дату
df_gr=df_full.groupby('created_at', as_index=False).count()[['created_at','id']]

# проверка
df_gr.iloc[:2]


,created_at,id
0,2022-06-16,1
1,2022-06-21,1


In [ ]:
# рисуем график
fig = px.bar(x=df_gr.created_at.to_list(),
             y=df_gr.id.to_list(), 
             labels={'x':'Дата публикации вакансии', 'y':'Количество публикаций'})
fig.show()


___________________________________________________________________________
> 5. Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
___________________________________________________________________________


In [ ]:
# добавляем в df_full колонку с днями недели  публикации 
df_full.insert(4, "weekday", 'null')

# проверка 
df_full.iloc[:2]


,id,alternate_url,area,created_at,weekday,employer,name,snippet,skills
0,66893967,https://hh.ru/vacancy/66893967,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-06-16 00:00:00,null,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Старший менеджер по обслуживанию,{'requirement': 'Хочешь стать экспертом в банк...,[{'name': 'Банковское кассовое обслуживание'}]
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01 00:00:00,null,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."


In [ ]:
# добавляем в df_full данные по дням недели и 
for i in range(len(df_full)):
    df_full.weekday[i]= df_full.created_at[i].strftime("%A")

# проверка
df_full.iloc[:2]


,id,alternate_url,area,created_at,weekday,employer,name,snippet,skills
0,66893967,https://hh.ru/vacancy/66893967,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-06-16 00:00:00,Thursday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Старший менеджер по обслуживанию,{'requirement': 'Хочешь стать экспертом в банк...,[{'name': 'Банковское кассовое обслуживание'}]
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01 00:00:00,Friday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."


In [ ]:
# подсчитываем количество публикаций по дням 
df_gr_weekday=df_full.groupby('weekday', as_index=False).count()[['weekday','id']]

# проверка
df_gr_weekday.iloc[:10]

,weekday,id
0,Friday,4
1,Monday,1
2,Thursday,8
3,Tuesday,5
4,Wednesday,2


In [ ]:
# ищем название дня, в который идёт самая активная публикация вакансий

max_element = df_gr_weekday['id'].max()
print('Максимальное количество публикаций равно ',max_element,', по ',max_element,'вакансий были опубликованы в ' )
for i in range(len(df_gr_weekday)):
    if df_gr_weekday.id[i]== max_element:print('-', df_gr_weekday.weekday[i])

Максимальное количество публикаций равно  8 , по  8 вакансий были опубликованы в 
- Thursday


---
> 6. Найдите те вакансии с использованием python, которые вам интересны
---



In [ ]:
# Формируем новый датафрейм с выборкой
df_sort=df_full[df_full['snippet'].str.contains('Python') | df_full['skills'].astype(str).str.contains('Python')| df_full['skills'].astype(str).str.contains('Публичные')] 

# навык "Публичные" добавлен для увеличения выборки - для выполнения задания неоходимо данную запись убрать 
# проверяем
df_sort.iloc[:2]

,id,alternate_url,area,created_at,weekday,employer,name,snippet,skills
1,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01 00:00:00,Friday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."
4,67577858,https://hh.ru/vacancy/67577858,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-07 00:00:00,Thursday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Стажер Data Science,{'requirement': 'Хорошо знаешь мат. статистику...,[]


In [ ]:
# проводим переиндексацию для дальнейшей работы
df_sort.reset_index(drop=True, inplace=True)

# проверяем
df_sort.iloc[:2]

,id,alternate_url,area,created_at,weekday,employer,name,snippet,skills
0,46580388,https://hh.ru/vacancy/46580388,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-01 00:00:00,Friday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Клиентский менеджер прямых продаж,"{'requirement': None, 'responsibility': 'Клиен...","[{'name': 'Публичные выступления'}, {'name': '..."
1,67577858,https://hh.ru/vacancy/67577858,"{'id': '4', 'name': 'Новосибирск', 'url': 'htt...",2022-07-07 00:00:00,Thursday,"{'id': '3529', 'name': 'СБЕР', 'url': 'https:/...",Стажер Data Science,{'requirement': 'Хорошо знаешь мат. статистику...,[]


---
> 7. Определите по полю skills какие навыки больше всего востребованы для этих вакансий
---


In [275]:
# создаем пустой датафрейм 
df_skills_sort = pd.DataFrame(columns =['name','count_skills'])
# проверяем результат
df_skills_sort.iloc[:2]

,name,count_skills


In [276]:
# в цикле заполнеям дата фрейм разобранными значениями поля skills
for i in range(len(df_sort)):  
    skills=df_sort['skills'].astype(str).loc[i]
    print(skills)
    for j in range(skills.count("name")): 
        new_row_1=skills.partition("'name': '")[2]
        new_row_2=new_row_1.partition("'")[0]
        new_row = {'name':new_row_2, 'count_skills':1}
        df_skills_sort=df_skills_sort.append(new_row, ignore_index=True)
        skills=new_row_1.partition("'")[2]
        
# проверяем результат
df_skills_sort.iloc[:10]

[{'name': 'Публичные выступления'}, {'name': 'Прямые продажи'}, {'name': 'Холодные звонки'}, {'name': 'Активные продажи'}, {'name': 'Поиск и привлечение клиентов'}]
[]


,name,count_skills
0,Публичные выступления,1
1,Прямые продажи,1
2,Холодные звонки,1
3,Активные продажи,1
4,Поиск и привлечение клиентов,1


In [277]:
# группируем данные
df_gr_skills=df_skills_sort.groupby('name', as_index=False).count()[['name','count_skills']]

# проверка
df_gr_skills.iloc[:10]


,name,count_skills
0,Активные продажи,1
1,Поиск и привлечение клиентов,1
2,Прямые продажи,1
3,Публичные выступления,1
4,Холодные звонки,1


In [280]:
# выбираем максимальное количество вхождений 
max_element = df_gr_skills['count_skills'].max()
print('Максимальное количество повторов навыка равно ',max_element,', для вакансий требуються навыки ')

for i in range(len(df_gr_skills)):
    if df_gr_skills.count_skills[i]== max_element: print('-', df_gr_skills.name[i])

Максимальное количество повторов навыка равно  1 , для вакансий требуються навыки 
- Активные продажи
- Поиск и привлечение клиентов
- Прямые продажи
- Публичные выступления
- Холодные звонки


---
> 8. Постройте график наиболее востребованных навыков для аналитика данных
---


In [283]:
# считаем наиболеевостребованным навыки с числом повтрений максимум -3 (в моей выборке таких нет, но пишем общий алгоритм)

df_gr_skills_sort=df_gr_skills[df_gr_skills['count_skills']>= df_gr_skills['count_skills'].max()-3] 

# рисуем график
fig = px.bar(x=df_gr_skills_sort.name.to_list(),
             y=df_gr_skills_sort.count_skills.to_list(), 
             labels={'x':'Навык', 'y':'Количество вакансий с требованием навыка'})
fig.show()